In [5]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_only import*
MP_param_file = os.path.join(root_path,'iltransr/pre-trained params/MP_best.params')

In [6]:
cnn_dropout = 0.1
batch_size = 64
bucket_num, bucket_ratio = 2, 0

In [ ]:
net = ILNet(dropout=cnn_dropout)
net.encoder = model.encoder
net.src_embed =  model.src_embed
net.hybridize()

In [8]:
net.load_parameters(MP_param_file,ctx=ctx)
net.initialize(init=mx.init.Xavier(magnitude=magnitude), ctx=ctx)

In [10]:
import pandas as pd
to_predict_data = pd.read_excel('smiles.xlsx')

In [11]:
predict_smiles = to_predict_data['SMILES'].map(canonical_smile).map(no_split).map(preprocess)
predict_smiles_lengths = to_predict_data['SMILES'].map(canonical_smile).map(no_split).map(get_length)
predict_sigma =mx.nd.ones((len(predict_smiles),1))
predict_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(predict_smiles,predict_sigma))
predict_dataloader = get_predict_dataloader(predict_dataset,predict_smiles_lengths,batch_size=batch_size, bucket_num=bucket_num,bucket_ratio=bucket_ratio)

In [12]:
predicted = predict(net, predict_dataloader,ctx)

[02:47:08] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable
[02:47:09] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable


In [13]:
df = pd.DataFrame(predicted)
df.to_csv('predicted_MP.csv')
print(df)

               0
0     411.611969
1     462.203979
2     303.537567
3     290.284790
4     464.059723
...          ...
5513  441.880371
5514  430.401062
5515  347.083405
5516  526.852844
5517  438.175537

[5518 rows x 1 columns]
